# Regression on Yb based OLED dataset by LightAutoML

Designed by Koshelev Daniil

https://github.com/Lamblador/Yb_OLED_Dataset

This noteboot show how to read and use YbOLED dataset with LightAutoML package.

# Import and installation of modules

In [1]:
!pip install lightautoml
from IPython.display import clear_output
clear_output()

Due to the pandas and numpy 2.0.2 conflict is is nessesary to reinstall and **reconnect** to the notebook.

In [2]:
!pip install --upgrade --force-reinstall numpy==1.26.4
# Check
import numpy as np
print(np.__version__)  # 1.26.4

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
1.26.4


In [3]:
import pandas as pd
import numpy as np
import torch
import sklearn

In [4]:
!git clone https://github.com/Lamblador/Yb_OLED_Dataset/

Cloning into 'Yb_OLED_Dataset'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 44 (delta 17), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (44/44), 410.42 KiB | 4.77 MiB/s, done.
Resolving deltas: 100% (17/17), done.


# Initializing of the LightAutoML

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Standard python libraries
import os
import requests
# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch
# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco, ReportDecoUtilized
from lightautoml.addons.tabular_interpretation import SSWARM

/usr/local/lib/python3.11/dist-packages/lightautoml/transformers/text.py:22: UserWarning: 'gensim' - package isn't installed
  warnings.warn("'gensim' - package isn't installed")


# Fucntion declaration

In [19]:
import os
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from typing import Tuple, Dict, List, Optional
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.inspection import partial_dependence
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task


# Constants
TARGETS = {
    'EQE': 'EQE, %',
    'ECE': 'ECE, uW/W',
    'Uon': 'Uon',
    'MaxIrradiance': 'Max. irradiance, uW/cm2'
}
COLS_TO_DROP = [
    'year', 'Doi', 'number', 'IUPAC anionic', 'IUPAC neutral',
    'CAS anionic', 'CAS neutral', 'SMILES anionic', 'SMILES neutral',
    'Method of EML mobility measurements', 'EML total, cm2/Vs',
    'Pixel size, mm2'
]


def prepare_dataset(
    target_key: str,
    file_path: str,
    test_size: float = 0.2,
    random_state: int = 42
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Load and preprocess dataset, splitting into train/test sets.

    Args:
        target_key: Key from TARGETS dictionary specifying the target variable
        file_path: Path to CSV data file
        test_size: Proportion of dataset to use for testing (0-1)
        random_state: Random seed for reproducibility

    Returns:
        Tuple of (train_data, test_data) DataFrames
    """
    # Load data and clean
    data = pd.read_csv(file_path, delimiter=';')
    data = data.drop(columns=COLS_TO_DROP, errors='ignore')

    # Handle targets
    target_col = TARGETS[target_key]
    other_targets = [v for k, v in TARGETS.items() if k != target_key]
    data = data.drop(columns=other_targets, errors='ignore')
    data = data.dropna(subset=[target_col])

    # Split data
    train_data, test_data = train_test_split(
        data,
        test_size=test_size,
        random_state=random_state
    )
    print(f"Dataset split: Train {train_data.shape}, Test {test_data.shape}")
    return train_data, test_data


def train_automl_model(
    train_data: pd.DataFrame,
    target_key: str,
    timeout: int = 300,
    n_threads: int = 4,
    n_folds: int = 5,
    random_state: int = 42
) -> TabularAutoML:
    """
    Train LightAutoML model on tabular data.

    Args:
        train_data: Training DataFrame
        target_key: Key from TARGETS dictionary specifying target variable
        timeout: Maximum training time in seconds
        n_threads: Number of CPU threads to use
        n_folds: Number of cross-validation folds
        random_state: Random seed

    Returns:
        Trained TabularAutoML model
    """
    # Configure AutoML task
    task = Task('reg', metric='mae')
    automl = TabularAutoML(
        task=task,
        timeout=timeout,
        cpu_limit=n_threads,
        reader_params={'n_jobs': n_threads, 'cv': n_folds, 'random_state': random_state},
        general_params={'use_algos': [['lgb', 'lgb_tuned', 'cb', 'cb_tuned']]}
    )

    # Define data roles
    roles = {
        'target': TARGETS[target_key],
        'numeric': [
            'HIL t, nm', 'HIL, HOMO', 'HIL, LUMO', 'HIL_HTL, HOMO',
            'HIL_HTL, LUMO', 'HTL tikness, nm', 'HTL, HOMO', 'HTL, LUMO',
            'HTL_EML, HOMO', 'HTL_EML, LUMO', 'EML tikness, nm', 'HOMO', 'LUMO',
            'EML_Matrix, HOMO', 'EML_Matrix, LUMO', 'EML/Matrix',
            'Matrix HOMO', 'Matrix LUMO', 'EML_ETL, HOMO',
            'EML_ETL, LUMO', 'ETL1, nm', 'ETL1, HOMO', 'ETL1, LUMO',
            'ETL1_ETL2, HOMO', 'ETL1_ETL2, LUMO', 'ETL2, nm', 'ETL2, HOMO',
            'ETL2, LUMO', 'ETL2_EIL, LUMO', 'EIL, nm', 'LUMO.1',
            'EIL_Cath, LUMO', 'nm', 'LUMO.2', 'nm.1',
            'LUMO.3', 't, us', 'QY, %', 'HOMO.1',
            'LUMO.4', 'EML hole mobilty, cm2/Vs',
            'EML electron mobility, cm2/Vs', 'EML total, cm2/Vs'
        ],
        'text': [
            'HIL', 'HTL', 'Matrix', 'EML method', 'ETL1', 'ETL2', 'EIL',
            'Cathode1', 'Cathode', 'Method (HOMO/LUMO)'
        ]
    }

    automl.fit_predict(train_data, roles=roles, verbose=1)
    return automl


def validate_model(
    model: TabularAutoML,
    test_data: pd.DataFrame,
    target_key: str
) -> Dict[str, float]:
    """
    Evaluate model performance on test data.

    Args:
        model: Trained AutoML model
        test_data: Test DataFrame
        target_key: Key from TARGETS dictionary specifying target variable

    Returns:
        Dictionary of evaluation metrics (MAE, MAPE, R2)
    """
    target_col = TARGETS[target_key]
    y_true = test_data[target_col]
    y_pred = model.predict(test_data).data[:, 0]

    return {
        'MAE': mean_absolute_error(y_true, y_pred),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred),
        'R2': r2_score(y_true, y_pred)
    }


def save_artifacts(
    model: TabularAutoML,
    test_data: pd.DataFrame,
    target_key: str,
    test_size: float,
    n_folds: int,
    output_dir: str = "outputs"
) -> None:
    """
    Save model artifacts including model, feature importance, and PDP plots.

    Args:
        model: Trained AutoML model
        test_data: Test DataFrame for feature analysis
        target_key: Target variable key for naming
        test_size: Test size used for naming
        n_folds: Number of CV folds used for naming
        output_dir: Output directory path
    """
    os.makedirs(output_dir, exist_ok=True)
    base_name = f"{target_key}_TS{test_size}_NF{n_folds}"

    # Save model
    model_path = os.path.join(output_dir, f"{base_name}_model.pkl")
    joblib.dump(model, model_path)

    # Save feature importance
    fi = model.get_feature_scores('accurate', test_data, silent=True)
    fi_path = os.path.join(output_dir, f"{base_name}_fi.csv")
    fi.to_csv(fi_path, index=False)

    # # Generate PDP plots
    # top_features = fi.head(3)['Feature'].tolist()
    # for i, feature in enumerate(top_features, 1):
    #     pdp, axes = partial_dependence(
    #         model, test_data, features=[feature], kind='average'
    #     )
    #     plt.figure()
    #     plt.plot(axes[0], pdp[0])
    #     plt.title(f"Partial Dependence: {feature}")
    #     plt.savefig(os.path.join(output_dir, f"{base_name}_pdp_{i}.png"))
    #     plt.close()


def run_experiment(
    target_key: str,
    file_path: str,
    test_sizes: List[float] = [0.1, 0.2, 0.3],
    n_folds_list: List[int] = [4, 5, 6, 7],
    timeout: int = 300,
    n_threads: int = 4,
    random_state: int = 42
) -> pd.DataFrame:
    """
    Run full experiment with hyperparameter grid search.

    Args:
        target_key: Target variable key to optimize for
        file_path: Path to data file
        test_sizes: List of test sizes to evaluate
        n_folds_list: List of CV folds to evaluate
        timeout: Training timeout per model
        n_threads: CPU threads to use
        random_state: Random seed

    Returns:
        DataFrame with metrics from all experiments
    """
    all_metrics = []

    for test_size in test_sizes:
        for n_folds in n_folds_list:
            # Prepare data
            train, test = prepare_dataset(
                target_key, file_path, test_size, random_state
            )

            # Train model
            model = train_automl_model(
                train, target_key, timeout, n_threads, n_folds, random_state
            )

            # Validate and save
            metrics = validate_model(model, test, target_key)
            metrics.update({
                'test_size': test_size,
                'n_folds': n_folds,
                'target': target_key
            })
            all_metrics.append(metrics)

            save_artifacts(model, test, target_key, test_size, n_folds)

    return pd.DataFrame(all_metrics)


# Example usage
if __name__ == "__main__":
    results = run_experiment(
        target_key='Uon', #'EQE''ECE''Uon''MaxIrradiance'
        file_path='/content/Yb_OLED_Dataset/yb_oled_data_short_ML.csv',
        test_sizes=[0.1, 0.15, 0.2],
        n_folds_list=[4, 5, 6, 7]
    )
    results.to_csv("experiment_results.csv", index=False)

Dataset split: Train (59, 53), Test (7, 53)
[08:01:40] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[08:01:40] Task: reg



INFO:lightautoml.automl.presets.base:Task: reg



[08:01:40] Start automl preset with listed constraints:


...

INFO:lightautoml.automl.blend:Blending: best score = -1.0820704, best weights = [0.         0.         0.7777139  0.22228608]


[08:18:33] Automl preset training completed in 75.05 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 75.05 seconds



[08:18:33] Model description:
Final prediction for new objects (level 0) = 
	 0.77771 * (7 averaged models Lvl_0_Pipe_0_Mod_2_CatBoost) +
	 0.22229 * (7 averaged models Lvl_0_Pipe_0_Mod_3_Tuned_CatBoost) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.77771 * (7 averaged models Lvl_0_Pipe_0_Mod_2_CatBoost) +
	 0.22229 * (7 averaged models Lvl_0_Pipe_0_Mod_3_Tuned_CatBoost) 



In [10]:
from google.colab import files as colab_files  # Переименуем модуль для ясности
import zipfile
import os

def download_folder_as_zip(folder_path, zip_name):
    """
    Compresses a folder in a ZIP archive and offers to download it

    Parameters:
        Folder path (str): The path to the folder that needs to be blocked
        postal code name (str): I'm for a ZIP archive (without registration .zip)
    """
    # Creating zip-archive
    zip_filename = f"{zip_name}.zip"
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files_in_dir in os.walk(folder_path):
            for file in files_in_dir:
                file_path = os.path.join(root, file)
                # Addition files into the zip
                arcname = os.path.relpath(file_path, os.path.dirname(folder_path))
                zipf.write(file_path, arcname)

    # Download zip
    colab_files.download(zip_filename)

In [20]:
download_folder_as_zip('/content/outputs', 'experiment_results_zip_4_7_ECE')
colab_files.download('/content/experiment_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>